You are looking at data from an e-commerce website. The site is very simple and has just 4 pages:
1. The ﬁrst page is the home page. When you come to the site for the ﬁrst time, you can only land on the home page as a ﬁrst page.
2. From the home page, the user can perform a search and land on the search page.
3. From the search page, if the user clicks on a product, she will get to the payment page, where she is asked to provide payment information in order to buy that product.
4. If she does decide to buy, she ends up on the conﬁrmation page

The company CEO isn't very happy with the volume of sales and, especially, of sales coming from new users. Therefore, she asked you to investigate whether there is something wrong in the conversion funnel or, in general, if you could suggest how conversion rate can be improved.

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

# Index

# Load the data

In [64]:
allusers = pd.read_csv("home_page_table.csv",index_col="user_id")
users_to_search = pd.read_csv("search_page_table.csv",index_col="user_id")
users_to_pay = pd.read_csv("payment_page_table.csv",index_col="user_id")
users_to_confirm = pd.read_csv("payment_confirmation_table.csv",index_col="user_id")

In [65]:
allusers.head()

,page
user_id,
313593,home_page
468315,home_page
264005,home_page
290784,home_page
639104,home_page


In [66]:
print(users_to_search.head())
print(users_to_pay.head())
print(users_to_confirm.head())

                page
user_id             
15866    search_page
347058   search_page
577020   search_page
780347   search_page
383739   search_page
                 page
user_id              
253019   payment_page
310478   payment_page
304081   payment_page
901286   payment_page
195052   payment_page
                              page
user_id                           
123100   payment_confirmation_page
704999   payment_confirmation_page
407188   payment_confirmation_page
538348   payment_confirmation_page
841681   payment_confirmation_page


In [67]:
allusers.loc[users_to_search.index,"page"] = users_to_search.page
allusers.loc[users_to_pay.index,"page"] = users_to_pay.page
allusers.loc[users_to_confirm.index,"page"] = users_to_confirm.page

In [68]:
allusers.head()

,page
user_id,
313593,home_page
468315,home_page
264005,home_page
290784,home_page
639104,search_page


In [69]:
# give it a better, more clear name
allusers.rename(columns={'page':'final_page'},inplace=True)

In [70]:
allusers['final_page'].value_counts()

home_page                    45200
search_page                  39170
payment_page                  5578
payment_confirmation_page      452
Name: final_page, dtype: int64

In [71]:
allusers.head()

,final_page
user_id,
313593,home_page
468315,home_page
264005,home_page
290784,home_page
639104,search_page


In [72]:
# change string to ordered-categorical feature
# pages = ["home_page","search_page","payment_page","payment_confirmation_page"]
allusers["final_page"] = allusers.final_page.astype("category")

In [73]:
allusers.head()

,final_page
user_id,
313593,home_page
468315,home_page
264005,home_page
290784,home_page
639104,search_page


In [75]:
user_infos = pd.read_csv("user_table.csv",index_col="user_id",parse_dates=['date'])


In [76]:
user_infos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 450007 to 438929
Data columns (total 3 columns):
date      90400 non-null datetime64[ns]
device    90400 non-null object
sex       90400 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 2.8+ MB


In [77]:
user_infos.head()

,date,device,sex
user_id,,,
450007,2015-02-28,Desktop,Female
756838,2015-01-13,Desktop,Male
568983,2015-04-09,Desktop,Male
190794,2015-02-18,Desktop,Female
537909,2015-01-15,Desktop,Male


In [78]:
allusers = allusers.join(user_infos)
allusers.head()

,final_page,date,device,sex
user_id,,,,
313593,home_page,2015-02-26,Desktop,Female
468315,home_page,2015-02-21,Desktop,Male
264005,home_page,2015-03-25,Desktop,Female
290784,home_page,2015-03-14,Desktop,Male
639104,search_page,2015-01-03,Desktop,Female


In [80]:
allusers.to_csv("all_users.csv",index_label="user_id")

# Answer question 1
<span style='color:blue;font-size:1.2em'>A full picture of funnel conversion rate for both desktop and mobile</span>

In [81]:
def conversion_rates(df):
    stage_counts = df.final_page.value_counts()
    # #users converts from current page
    convert_from = stage_counts.copy()

    total = df.shape[0]
    for page in stage_counts.index:
        n_left = stage_counts.loc[page]# how many users just stop at current page
        n_convert = total - n_left
        convert_from[page] = n_convert
        total = n_convert

    cr = pd.concat([stage_counts,convert_from],axis=1,keys=["n_drop","n_convert"])
    cr["convert_rates"] = cr.n_convert.astype(np.float)/(cr.n_drop + cr.n_convert)
    cr['drop_rates'] = 1 - cr.convert_rates

    return cr

In [82]:
allusers.groupby('device').apply(conversion_rates)

n_drop  n_convert  convert_rates  \
device                                                                
Desktop home_page                   30100      30100       0.500000   
        search_page                 27090       3010       0.100000   
        payment_page                 2860        150       0.049834   
        payment_confirmation_page     150          0       0.000000   
Mobile  home_page                   15100      15100       0.500000   
        search_page                 12080       3020       0.200000   
        payment_page                 2718        302       0.100000   
        payment_confirmation_page     302          0       0.000000   

                                   drop_rates  
device                                         
Desktop home_page                    0.500000  
        search_page                  0.900000  
        payment_page                 0.950166  
        payment_confirmation_page    1.000000  
Mobile  home_page                    0.500000  
        search_page                  0.800000  
        payment_page                 0.900000  
        payment_confirmation_page    1.000000

In [83]:
allusers.groupby('device')['final_page'].apply(lambda s: s.value_counts(normalize=True)).unstack()

,home_page,search_page,payment_page,payment_confirmation_page
device,,,,
Desktop,0.5,0.45,0.047508,0.002492
Mobile,0.5,0.40,0.090000,0.010000


# Answer question 2
<span style='color:blue;font-size:1.2em'>Some insights on what the product team should focus on in order to improve conversion rate as well as anything you might discover that could help improve conversion rate.</span>

In [84]:
allusers.head()

,final_page,date,device,sex
user_id,,,,
313593,home_page,2015-02-26,Desktop,Female
468315,home_page,2015-02-21,Desktop,Male
264005,home_page,2015-03-25,Desktop,Female
290784,home_page,2015-03-14,Desktop,Male
639104,search_page,2015-01-03,Desktop,Female


In [85]:
X = allusers.copy()

In [86]:
X.device.value_counts()

Desktop    60200
Mobile     30200
Name: device, dtype: int64

In [87]:
X['from_mobile'] = (X.device == 'Mobile').astype(int)
del X['device']

In [88]:
X['is_male'] = (X.sex == 'Male').astype(int)
del X['sex']

In [89]:
X['converted'] = (X.final_page == 'payment_confirmation_page').astype(int)
del X['final_page']

In [90]:
X.converted.mean()# a highly imbalanced classification problem

0.005

## Impact of date

In [91]:
X.date.describe()

count                   90400
unique                    120
top       2015-02-08 00:00:00
freq                      877
first     2015-01-01 00:00:00
last      2015-04-30 00:00:00
Name: date, dtype: object

In [92]:
X['weekday'] = X.date.dt.weekday_name
del X['date']

In [93]:
X.head()

,from_mobile,is_male,converted,weekday
user_id,,,,
313593,0,0,0,Thursday
468315,0,1,0,Saturday
264005,0,0,0,Wednesday
290784,0,1,0,Saturday
639104,0,0,0,Saturday


In [96]:
X.groupby('weekday')['converted'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
weekday,,
Monday,12955,0.005944
Saturday,12801,0.005468
Tuesday,12697,0.005434
Thursday,13444,0.004909
Sunday,12922,0.004798
Friday,12750,0.004235
Wednesday,12831,0.004209


## Impact of gender

In [97]:
X.groupby('is_male')['converted'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
is_male,,
0,45075,0.005347
1,45325,0.004655


## Statistical Test

In [98]:
X = pd.get_dummies(X,prefix='',prefix_sep='')
X.head()

,from_mobile,is_male,converted,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
user_id,,,,,,,,,,
313593,0,0,0,0,0,0,0,1,0,0
468315,0,1,0,0,0,1,0,0,0,0
264005,0,0,0,0,0,0,0,0,0,1
290784,0,1,0,0,0,1,0,0,0,0
639104,0,0,0,0,0,1,0,0,0,0


In [99]:
y = X.converted
X = X.loc[:,X.columns != 'converted']

In [100]:
scores, pvalues = chi2(X,y)

In [102]:
pd.DataFrame({'chi2_score':scores,'chi2_pvalue':pvalues},index=X.columns).sort_values(by='chi2_score',ascending=False)

,chi2_pvalue,chi2_score
from_mobile,7.153913e-35,151.758794
Monday,1.278169e-01,2.318821
Wednesday,2.037208e-01,1.615497
Friday,2.208761e-01,1.498670
is_male,2.980938e-01,1.082703
Saturday,4.525166e-01,0.564341
Tuesday,4.877434e-01,0.481501
Sunday,7.447867e-01,0.105964
Thursday,8.814147e-01,0.022253


In [103]:
del X['Tuesday']# remove one redundant feature

In [104]:
dt = DecisionTreeClassifier(max_depth=3,min_samples_leaf=20,min_samples_split=20)
dt.fit(X,y)
export_graphviz(dt,feature_names=X.columns,class_names=['NotConvert','Converted'],
                proportion=True,leaves_parallel=True,filled=True)

'digraph Tree {\nnode [shape=box, style="filled", color="black"] ;\ngraph [ranksep=equally, splines=polyline] ;\n0 [label="from_mobile <= 0.5\\ngini = 0.01\\nsamples = 100.0%\\nvalue = [0.995, 0.005]\\nclass = NotConvert", fillcolor="#e58139fe"] ;\n1 [label="Wednesday <= 0.5\\ngini = 0.005\\nsamples = 66.6%\\nvalue = [0.998, 0.002]\\nclass = NotConvert", fillcolor="#e58139fe"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;\n2 [label="Sunday <= 0.5\\ngini = 0.005\\nsamples = 57.2%\\nvalue = [0.997, 0.003]\\nclass = NotConvert", fillcolor="#e58139fe"] ;\n1 -> 2 ;\n3 [label="gini = 0.005\\nsamples = 47.7%\\nvalue = [0.997, 0.003]\\nclass = NotConvert", fillcolor="#e58139fe"] ;\n2 -> 3 ;\n4 [label="gini = 0.004\\nsamples = 9.5%\\nvalue = [0.998, 0.002]\\nclass = NotConvert", fillcolor="#e58139fe"] ;\n2 -> 4 ;\n5 [label="is_male <= 0.5\\ngini = 0.004\\nsamples = 9.4%\\nvalue = [0.998, 0.002]\\nclass = NotConvert", fillcolor="#e58139ff"] ;\n1 -> 5 ;\n6 [label="gini = 0.002\